In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
cleaned_data = pd.read_csv(
    'Data\cleaned_dataset.csv',
    engine="python", 
    sep=None           # auto-detects delimiter
)

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\66432\AppData\Local\Temp\ipykernel_5636\2934003315.py:2: SyntaxWarning: invalid escape sequence '\c'
  'Data\cleaned_dataset.csv',


In [3]:
# Filter the data by process type, leave only the NIEUW requests
cleaned_data["Process_type"] = cleaned_data["Procestype"].map({
    "NIEUW": 1,
    "WIJZIGING": 0
})

In [4]:
data_NIEUW = cleaned_data[cleaned_data["Process_type"] == 1].copy()

In [5]:
col1 = "Aanvraag_Eerste_Locatie_Vroegste_Vertrektijd"

# Ensure datetimelike
data_NIEUW[col1] = pd.to_datetime(data_NIEUW[col1], errors="coerce")

# Extract hour (0–23)
data_NIEUW["Hour_group"] = data_NIEUW[col1].dt.hour

print(data_NIEUW[[col1, "Hour_group"]].head(10))

   Aanvraag_Eerste_Locatie_Vroegste_Vertrektijd  Hour_group
0                           1900-01-01 07:00:00         7.0
1                           1900-01-01 13:00:00        13.0
3                           1900-01-01 22:15:00        22.0
4                           1900-01-01 09:00:00         9.0
7                           1900-01-01 07:00:00         7.0
8                           1900-01-01 10:30:00        10.0
11                          1900-01-01 07:00:00         7.0
13                          1900-01-01 06:45:00         6.0
14                          1900-01-01 07:00:00         7.0
18                          1900-01-01 07:30:00         7.0


In [6]:
# Count total missing values in Hour_group
missing_count = data_NIEUW["Hour_group"].isna().sum()
print(f"Total missing Hour_group values: {missing_count}")

# Show the rows where Hour_group is missing
missing_rows = data_NIEUW[data_NIEUW["Hour_group"].isna()][
    ["Aanvraag_Eerste_Locatie_Vroegste_Vertrektijd", "Hour_group"]
]

print(missing_rows.head(20))  # show first 20 missing cases

Total missing Hour_group values: 301
      Aanvraag_Eerste_Locatie_Vroegste_Vertrektijd  Hour_group
2430                                           NaT         NaN
3324                                           NaT         NaN
3764                                           NaT         NaN
4620                                           NaT         NaN
4726                                           NaT         NaN
5304                                           NaT         NaN
5361                                           NaT         NaN
6991                                           NaT         NaN
7001                                           NaT         NaN
8625                                           NaT         NaN
9122                                           NaT         NaN
9452                                           NaT         NaN
9774                                           NaT         NaN
10378                                          NaT         NaN
10808             

In [7]:
# Filter out those rows without requested earliest departure time
data_NIEUW_filtered = data_NIEUW.dropna(subset=["Hour_group"]).copy()

In [8]:
# Encode the hours of day in a cyclical way, sothat 0:00 is close to 23:00
data_NIEUW_filtered["Hour_sin"] = np.sin(2 * np.pi * data_NIEUW_filtered["Hour_group"] / 24)
data_NIEUW_filtered["Hour_cos"] = np.cos(2 * np.pi * data_NIEUW_filtered["Hour_group"] / 24)

# Show example
print(data_NIEUW_filtered[["Hour_group", "Hour_sin", "Hour_cos"]].head(10))

    Hour_group  Hour_sin      Hour_cos
0          7.0  0.965926 -2.588190e-01
1         13.0 -0.258819 -9.659258e-01
3         22.0 -0.500000  8.660254e-01
4          9.0  0.707107 -7.071068e-01
7          7.0  0.965926 -2.588190e-01
8         10.0  0.500000 -8.660254e-01
11         7.0  0.965926 -2.588190e-01
13         6.0  1.000000  6.123234e-17
14         7.0  0.965926 -2.588190e-01
18         7.0  0.965926 -2.588190e-01


In [9]:
col2 = "Aanvraag_Indienmoment"

# Ensure datetimelike
data_NIEUW_filtered[col2] = pd.to_datetime(data_NIEUW_filtered[col2], errors="coerce")

# Extract hour (0–23)
data_NIEUW_filtered["Submission_hour_group"] = data_NIEUW_filtered[col2].dt.hour

print(data_NIEUW_filtered[[col2, "Submission_hour_group"]].head(10))

   Aanvraag_Indienmoment  Submission_hour_group
0    2022-12-10 23:58:19                     23
1    2022-12-11 00:00:04                      0
3    2022-12-11 00:22:38                      0
4    2022-12-11 01:05:34                      1
7    2022-12-11 03:41:08                      3
8    2022-12-11 03:03:56                      3
11   2022-12-11 05:42:53                      5
13   2022-12-11 06:39:20                      6
14   2022-12-11 06:46:52                      6
18   2022-12-11 07:00:27                      7


In [10]:
# Map the reaction by prorail into binary numbers
data_NIEUW_filtered["Approved"] = data_NIEUW_filtered["Reactie_type"].map({
    "AANGEBODEN": 1,
    "GEEN_AANBIEDING_MOGELIJK": 0
})

In [11]:
# Copy the information of stabling and time tolerance from the corresponding columns
data_NIEUW_filtered["Stabling"] = data_NIEUW_filtered["Aanvraag_Laatste_Locatie_Opstellen_True"]
data_NIEUW_filtered["Tolerance"] = data_NIEUW_filtered["Aanvraag_time_range_Eerste_Locatie_Vertrektijd"]

In [12]:
# Encode the days of week in a cyclical way, so that Monday is close to Sunday
mapping = {"Monday":0, "Tuesday":1, "Wednesday":2, 
           "Thursday":3, "Friday":4, "Saturday":5, "Sunday":6}
data_NIEUW_filtered["Day_num"] = data_NIEUW_filtered["Aanvraag_Day"].map(mapping)
data_NIEUW_filtered["Day_sin"] = np.sin(2 * np.pi * data_NIEUW_filtered["Day_num"] / 7)
data_NIEUW_filtered["Day_cos"] = np.cos(2 * np.pi * data_NIEUW_filtered["Day_num"] / 7)

#Using OneHotEncoder to encode the origin and destination of a request
encoder_origin = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
origin_encoded = encoder_origin.fit_transform(data_NIEUW_filtered[["Aanvraag_Eerste_Locatie_Dienstregelpunt"]])
origin_columns = encoder_origin.get_feature_names_out(["Aanvraag_Eerste_Locatie_Dienstregelpunt"])
origin_df = pd.DataFrame(origin_encoded, columns=origin_columns, index=data_NIEUW_filtered.index)

encoder_dest = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
dest_encoded = encoder_dest.fit_transform(data_NIEUW_filtered[["Aanvraag_Laatste_Locatie_Dienstregelpunt"]])
dest_columns = encoder_dest.get_feature_names_out(["Aanvraag_Laatste_Locatie_Dienstregelpunt"])
dest_df = pd.DataFrame(dest_encoded, columns=dest_columns, index=data_NIEUW_filtered.index)

# Integrate all the columns into a numeric dataset for model processing
numeric_dataset = pd.concat([data_NIEUW_filtered[["Process_type"]], 
                             data_NIEUW_filtered[["Stabling"]], 
                             data_NIEUW_filtered[["Tolerance"]],
                             data_NIEUW_filtered[["Day_sin"]],
                             data_NIEUW_filtered[["Day_cos"]], 
                             data_NIEUW_filtered[["Hour_sin"]],
                             data_NIEUW_filtered[["Hour_cos"]],
                             origin_df, 
                             dest_df, 
                             data_NIEUW_filtered[["Approved"]]], 
                             axis=1)

print(numeric_dataset.head())

   Process_type  Stabling  Tolerance   Day_sin   Day_cos  Hour_sin  Hour_cos  \
0             1         0       30.0 -0.974928 -0.222521  0.965926 -0.258819   
1             1         0        5.0 -0.781831  0.623490 -0.258819 -0.965926   
3             1         0        5.0 -0.781831  0.623490 -0.500000  0.866025   
4             1         0       15.0 -0.781831  0.623490  0.707107 -0.707107   
7             1         0       15.0 -0.781831  0.623490  0.965926 -0.258819   

   Aanvraag_Eerste_Locatie_Dienstregelpunt_Ac  \
0                                         0.0   
1                                         0.0   
3                                         0.0   
4                                         0.0   
7                                         0.0   

   Aanvraag_Eerste_Locatie_Dienstregelpunt_Ah  \
0                                         0.0   
1                                         0.0   
3                                         0.0   
4                           

In [ ]:
# Save to new CSV file
numeric_dataset.to_csv("numeric_dataset.csv", index=False)